# Study on public transport
This notebook uses datas from [opentransportdata](https://opentransportdata.swiss/en/) to get some information about swiss trainstation and line quality.<br><br>
![Logo he-arc](./Image/hearclogo.png)

#### Authors
- David Schnegg
- Isaac Metthez

© HE-ARC, all rights reserved

## Introduction

Swiss has one of the best railway system in the world, but sometimes it fails and we were interested to find how many times.
We are aware that such things already do exist, but we wanted make ours. 
CFF/SBB Provied a extended API so we based our statistical analysis on the delay got by subtracting the real time to predicted time.
Then we ploted the result to show it in a comprensive and interactive way.
Sadly we dicovered that Lausanne isn't the worst city when it comes to trains punctuality.

![Swiss punctuality by kanton](./swiss_transport_punctuality.svg)

## Python's Packages

### Install
We use differents packages to run our project:

You can install them by executing next cell :<br>
Note : (you may need to restart the kernel to use updated packages)

In [ ]:
pip install pandas lxml ipywidgets pygal pygal_maps_ch matplotlib requests 

### Import 
Since the packages are installed, we can import them.

In [ ]:
import lxml.html
import io
import pygal
import pygal.style
import pygal.maps
import datetime
import pandas as pd
import IPython as IPy
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import time 
import timer
import requests
import os
import zipfile

## Data acquisition

### Swiss public transport
As mentionned before we use datas from [opentransportdata](https://opentransportdata.swiss/en/) and more specifically this [page](https://opentransportdata.swiss/fr/dataset/istdaten/) to get swiss public transport datas.<br>
After executing the following cell you will be able to select range of datas to download (maximum 50 days, minimum 1 day). <br>
Note : (Be carefull one day data size is near to 500MB.)

In [ ]:
# download the page where are the links for daily data
req = "https://opentransportdata.swiss/fr/dataset/istdaten"
page = requests.get(req)
webpage = lxml.html.fromstring(page.content)

# get links in webpage
links = webpage.xpath("//a[contains(@href,'https://opentransportdata.swiss/dataset/')]/@href")
options = [datetime.datetime.strptime(f'{i}'[-23:-13],'%Y-%m-%d').date().strftime("%d/%m/%Y") for i in links[::-1]]


def predict_data_size(range):
    size_indicator.value = ((1+
                            options[::-1].index(range[0])-
                            options[::-1].index(range[1]))
                            *500)

def download():
    try:
        timeStarted = time.perf_counter()
        start =  options[::-1].index(time_range_data_slider.get_interact_value()[1])
        end = options[::-1].index(time_range_data_slider.get_interact_value()[0])
        allContent = bytes()
        for link in links[start:end+1]:
            print("loading :")
            print(link)
            data = requests.get(link)
            allContent += data.content
    except:
        pass
    finally:
        os.makedirs("./Datas",exist_ok = True)
        open("./Datas/sbb_cff.csv","wb").write(allContent)  
        print(f"Datas downloaded in {time.perf_counter()-timeStarted:0.2f} seconds ")
 
# to call when interact
def on_widget_change(load:bool,range:tuple):
    if load == True:
        download_button.disabled = True
        time_range_data_slider.disabled = True
        download_button.value = False
        download()
        download_button.disabled = False
        time_range_data_slider.disabled = False
    else:
        predict_data_size(range)

# widget objects
time_range_data_slider = widgets.SelectionRangeSlider(
    options=options,
    index=(0, len(links)-1),
    description="Select Date",
    layout= widgets.Layout(width = "99%"),
    disabled=False
)
download_button =widgets.ToggleButton(
    value=False,
    description='Download',
    disabled=False,
    button_style='',
    tooltip='Description',
    icon='check'
)
size_indicator = widgets.FloatText(
    value=0,
    description='Size[MB]:',
    disabled=True,
    layout= widgets.Layout(width = "150px"),
)       

# widgets
interactions = widgets.interactive_output(on_widget_change,{'load':download_button,'range':time_range_data_slider} )
buttonLine =  widgets.HBox([size_indicator,download_button])
widgetBox = widgets.VBox([time_range_data_slider,buttonLine])
IPy.display.display(widgetBox, interactions)


### Swiss cities
For the last part we need to know in which kanton are the swiss cities located. To do that, we used at the beginning [ch-country-cities](https://simplemaps.com/static/data/country-cities/ch/ch.csv), but there was only 185 cities. That's why we now use [world-cities-simple](https://simplemaps.com/static/data/world-cities/basic/simplemaps_worldcities_basicv1.76.zip).<br>

In [ ]:
req = "https://simplemaps.com/static/data/world-cities/basic/simplemaps_worldcities_basicv1.76.zip"
zipped = requests.get(req)

with zipfile.ZipFile(io.BytesIO(zipped.content), mode= 'r') as zip:
    zip.extract('worldcities.csv','./Datas/')

## Load datas

### Load actual data csv
Load actual data, keep only the useful column, calculate  delays , create dictionnary from grouped.

In [ ]:
# Load sbb_cff data frame
df_actual_data = pd.read_csv(filepath_or_buffer="./Datas/sbb_cff.csv",delimiter=';',low_memory=False)
print(f'data frame brut shape is : {df_actual_data.shape}')

# keep useful for project
df_actual_data = (df_actual_data[[
    'HALTESTELLEN_NAME', 
     'LINIEN_TEXT', 
     'AB_PROGNOSE',
     'AN_PROGNOSE',
     'ABFAHRTSZEIT',
     'ANKUNFTSZEIT',
     'AN_PROGNOSE_STATUS',
     'PRODUKT_ID'
    ]])
print(f'data frame useful shape is : {df_actual_data.shape}')

# keep only data for train and with time wich is real
df_real_time_for_train = df_actual_data[(df_actual_data["PRODUKT_ID"] == "Zug") & (df_actual_data["AN_PROGNOSE_STATUS"] == "REAL")].copy()

# get time from string
df_real_time_for_train["AB_PROGNOSE"] = pd.to_datetime(df_real_time_for_train["AB_PROGNOSE"], format="%d.%m.%Y %H:%M:%S")
df_real_time_for_train["AN_PROGNOSE"] = pd.to_datetime(df_real_time_for_train["AN_PROGNOSE"], format="%d.%m.%Y %H:%M:%S")
df_real_time_for_train["ABFAHRTSZEIT"] = pd.to_datetime(df_real_time_for_train["ABFAHRTSZEIT"], format="%d.%m.%Y %H:%M")
df_real_time_for_train["ANKUNFTSZEIT"] = pd.to_datetime(df_real_time_for_train["ANKUNFTSZEIT"], format="%d.%m.%Y %H:%M")

# calculate delay
df_real_time_for_train["delay_train"] = (df_real_time_for_train["AB_PROGNOSE"] - df_real_time_for_train["ABFAHRTSZEIT"]).dt.total_seconds()

# remove what is know useless
df_city_line_delay = df_real_time_for_train[['HALTESTELLEN_NAME', 'LINIEN_TEXT', 'delay_train']]

# filter abnormal datas
df_city_line_delay = df_city_line_delay.dropna()
positive = df_city_line_delay.index[df_city_line_delay['delay_train'] >= 0]
df_city_line_delay = df_city_line_delay.loc[positive]

# group by city and get a dictionnary
grouped_by_trainstation = df_city_line_delay.groupby(['HALTESTELLEN_NAME'])['delay_train']
trainstation_delay_dict = grouped_by_trainstation.apply(list).to_dict()

# group by line and get a dictionnary
grouped_by_line = df_city_line_delay.groupby(['LINIEN_TEXT'])['delay_train']
line_delay_dict = grouped_by_line.apply(list).to_dict()

### load cities data csv
Keep only the Swiss cities and make a dictionnary with the cities as key and the administration name as value.<br>
And create a conversion dictionnary.

In [ ]:
# Use for cities to administration name
swiss_cities_dict = (pd.read_csv(
    filepath_or_buffer="./Datas/worldcities.csv",
    delimiter=',',low_memory=False)[['country','city', 'admin_name']]
    .groupby('country')
    .get_group('Switzerland')
    .groupby('city')['admin_name']
    .apply(list).to_dict())

# Use for administration name to abbreviated admin name
kt_name_conversion = {
    "Aargau"                :  "kt-ag",
    "Appenzell Innerrhoden" :  "kt-ai",
    "Appenzell Ausserrhoden":  "kt-ar",
    "Basel-Landschaft"      :  "kt-bl",
    "Basel-Stadt"           :  "kt-bs",
    "Bern"                  :  "kt-be",
    "Fribourg"              :  "kt-fr",
    "Genève"                :  "kt-ge",
    "Glarus"                :  "kt-gl",
    "Graubünden"            :  "kt-gr",
    "Jura"                  :  "kt-ju",
    "Luzern"                :  "kt-lu",
    "Neuchâtel"             :  "kt-ne",
    "Nidwalden"             :  "kt-nw",
    "Obwalden"              :  "kt-ow",
    "Sankt Gallen"          :  "kt-sg",
    "Schaffhausen"          :  "kt-sh",
    "Schwyz"                :  "kt-sz",
    "Solothurn"             :  "kt-so",
    "Thurgau"               :  "kt-tg",
    "Ticino"                :  "kt-ti",
    "Uri"                   :  "kt-ur",
    "Valais"                :  "kt-vs",
    "Vaud"                  :  "kt-vd",
    "Zug"                   :  "kt-zg",
    "Zürich"                :  "kt-zh" }

## Plot results

### Box plot

Select a city or a line with a combobox to display the delay with a boxplot up to 10 choices (to see median and quartil).<br>

In [ ]:
plotted = []
last_lines_cities_value = 'Cities'

def plot(outliers: bool):
    values = []
    labels = []
    delay_dict = trainstation_delay_dict if last_lines_cities_value == 'Cities' else line_delay_dict
    for s in plotted:
        values.append(delay_dict[s])
        labels.append(s if len(s) < 30 else s[0:30])
    if len(values) > 0:
        plt.figure(figsize=(3*len(plotted),5))
        plt.boxplot(x=values,
                    labels=labels,
                    showfliers=outliers,
                    widths=0.8
                    )
    plt.title('Train delay')
    plt.ylabel('Delay in [s]')
    plt.show()

def on_widget_change(outliers: bool, newcity:str, newline:str, removed_city:tuple, remove_all:bool, lines_cities:str):
    global last_lines_cities_value, plotted
    plot(outliers)

    if last_lines_cities_value != lines_cities:
        last_lines_cities_value = lines_cities
        if lines_cities == 'Lines':
            add_city_to_plotted.disabled = True
            add_line_to_plotted.disabled = False
        else:
            add_city_to_plotted.disabled = False
            add_line_to_plotted.disabled = True
        plotted.clear()


    if 0 < len(plotted):
        remove_all_button.disabled = False
        lines_or_cities_button.disabled = True
    else:
        remove_all_button.disabled = True
        lines_or_cities_button.disabled = False


    if newcity != '' or newline != '':
        if len(plotted) > 10:
            plotted.pop(0)
        if lines_cities == 'Lines':
            plotted.append(newline)
        else:
            plotted.append(newcity)
        add_line_to_plotted.value = ''
        add_city_to_plotted.value = ''

    if removed_city != ():
        [plotted.remove(removed)for removed in remove_from_plotted.value]
    if remove_all:
        plotted.clear()
        remove_all_button.value = False


    remove_from_plotted.options = plotted

# widget objects
show_outliers = widgets.Checkbox(
    value=False,
    description='Outliers',
    disabled=False,
    indent=False
)
add_line_to_plotted = widgets.Combobox(
    options = list(line_delay_dict.keys()),
    placeholder='Select line',
    description='Line :',
    ensure_option=True,
    disabled=True
)
add_city_to_plotted = widgets.Combobox(
    options = list(trainstation_delay_dict.keys()),
    placeholder='Select city',
    description='City :',
    ensure_option=True,
    disabled=False
)
remove_from_plotted = widgets.SelectMultiple(
    description='Remove :',
    ensure_option=True,
    disabled=False
)
remove_all_button = widgets.ToggleButton(
    value=False,
    description='Remove all',
    disabled=True,
    button_style='',
    tooltip='Description',
    icon='none'
)
lines_or_cities_button = widgets.RadioButtons(
    options=['Cities', 'Lines'],
    value=last_lines_cities_value,
    description='Plot for :',
    disabled=False,
)

# interact
interactions = widgets.interactive_output(
    on_widget_change,{'outliers':show_outliers,
                      'newcity':add_city_to_plotted,
                      'newline':add_line_to_plotted,
                      'removed_city':remove_from_plotted,
                      'remove_all':remove_all_button,
                      'lines_cities':lines_or_cities_button
                        } )
widgetBox = widgets.VBox([ lines_or_cities_button,
                            add_city_to_plotted, 
                            add_line_to_plotted,
                            widgets.HBox([remove_from_plotted,
                                         widgets.VBox([remove_all_button,show_outliers])])])
IPy.display.display(widgetBox, interactions)

### Ranking

Rank the best or the worst cities/lines for the selection.

In [ ]:
def on_widget_change(ranked: int, acceptable_time:float, best_worse:str, lines_cities:str):
    test = df_city_line_delay.copy()
    if lines_cities == 'Lines':
        not_acceptable_delay = df_city_line_delay.index[df_city_line_delay['delay_train'] > acceptable_time]
        grouped_not_acceptable_delay = test.loc[not_acceptable_delay].groupby(['LINIEN_TEXT'])['delay_train']
        quality = ((grouped_by_line.size() - grouped_not_acceptable_delay.size()).fillna(grouped_by_line.size())/grouped_by_line.size()).sort_values() * 100
    else:
        not_acceptable_delay = df_city_line_delay.index[df_city_line_delay['delay_train'] > acceptable_time]
        grouped_not_acceptable_delay = test.loc[not_acceptable_delay].groupby(['HALTESTELLEN_NAME'])['delay_train']
        quality = ((grouped_by_trainstation.size() - grouped_not_acceptable_delay.size()).fillna(grouped_by_trainstation.size())/grouped_by_trainstation.size()).sort_values() * 100
     
    if best_worse == 'Best':
        dict_to_print = quality[::-1].head(ranked).apply(str).to_dict()
    else:
        dict_to_print = quality.head(ranked).apply(str).to_dict()
    for k,v in dict_to_print.items():
        print(f'{k: <30} : {float(v):.3f}')

# widget objects
number_ranked = widgets.IntSlider(
    value=10,
    min=1,
    max=30,
    step=1,
    description='Ranks',
    layout= widgets.Layout(width = "99%"),
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
maximum_acceptable_time = widgets.FloatSlider(
    value=300.0,
    min=30,
    max=1800,
    step=0.1,
    description='Threshold[s]',
    layout= widgets.Layout(width = "99%"),
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

lines_or_cities_button = widgets.RadioButtons(
    options=['Cities', 'Lines'],
    value=last_lines_cities_value,
    description='Rank for :',
    disabled=False,
)

show_best_button = widgets.RadioButtons(
    options=['Best', 'Worse'],
    value='Best',
    description='Rank for :',
    disabled=False,
)


# interact
interactions = widgets.interactive_output(
    on_widget_change,{'ranked':number_ranked,
                      'acceptable_time':maximum_acceptable_time,
                      'best_worse':show_best_button,
                      'lines_cities':lines_or_cities_button
                     } )
buttonLine =  widgets.HBox([lines_or_cities_button, show_best_button ])
widgetBox = widgets.VBox([ number_ranked, maximum_acceptable_time, buttonLine])
IPy.display.display(widgetBox, interactions)

### Evaluate a trainstation

Choose a train station to know its ranking and punctuality.

In [ ]:
def on_widget_change( acceptable_time:float, rated:str):
    test = df_city_line_delay.copy()
    if rated == '':
        # abort useless calculation
        pass 
    elif rated in line_delay_dict:
        
        not_acceptable_delay = df_city_line_delay.index[df_city_line_delay['delay_train'] > acceptable_time]
        grouped_not_acceptable_delay = test.loc[not_acceptable_delay].groupby(['LINIEN_TEXT'])['delay_train']
        quality = ((grouped_by_line.size() - grouped_not_acceptable_delay.size()).fillna(grouped_by_line.size())/grouped_by_line.size()).sort_values() * 100

        print(rated)
        # accuracy is
        print (quality[rated])
        
        print (int(quality.rank(method='dense',ascending=False)[rated]))
        print (int(quality.rank(method='dense',ascending=False).max()))
        spot_selection.value = ''

    elif rated in trainstation_delay_dict:
        not_acceptable_delay = df_city_line_delay.index[df_city_line_delay['delay_train'] > acceptable_time]
        grouped_not_acceptable_delay = test.loc[not_acceptable_delay].groupby(['HALTESTELLEN_NAME'])['delay_train']
        quality = ((grouped_by_trainstation.size() - grouped_not_acceptable_delay.size()).fillna(grouped_by_trainstation.size())/grouped_by_trainstation.size()).sort_values() * 100
        rank = int(quality.rank(method='dense',ascending=False)[rated])
        nb_rank = int(quality.rank(method='dense',ascending=False).max())
        print(f'{rated} is the {rank} out of {nb_rank} with {quality[rated] :.1f} of trains who have less than {acceptable_time:.0f} [s] of delay.')
        # accuracy is
        
        print ()
        print ()
        spot_selection.value = ''
    else:
        pass


# widget objects
maximum_acceptable_time = widgets.FloatSlider(
    value=300.0,
    min=30,
    max=1800,
    step=0.1,
    description='Threshold[s]',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
spot_selection = widgets.Combobox(
    options = list(trainstation_delay_dict.keys()) + list(line_delay_dict.keys()),
    placeholder='Selection',
    description='Line or trainstation:',
    ensure_option=True,
    disabled=False
)

# interact
interactions = widgets.interactive_output(
    on_widget_change,{'acceptable_time':maximum_acceptable_time,
                      'rated':spot_selection
                     } )
widgetBox = widgets.VBox([spot_selection,maximum_acceptable_time])
IPy.display.display(widgetBox, interactions)

### Map

Plot the best and the worst kanton.

In [ ]:
# try to reliate trainstation to city and make a dataframe with
list_kanton_delay = {'kanton' : [], 'delay':[]}
for k,v in trainstation_delay_dict.items():
    try:
        # this scope can be improved
        res = swiss_cities_dict.get(k) or swiss_cities_dict[k.split()[0]] 
        for d in v:
            list_kanton_delay['kanton'].append(res[0])
            list_kanton_delay['delay'].append(d)
    except:
        pass
df_kanton_delay = pd.DataFrame(list_kanton_delay)
gp_kanton_delay = df_kanton_delay.groupby(['kanton'])['delay']

#styple for pygal        
custom_style = pygal.style.Style(
    background='transparent',
    plot_background='transparent',
    foreground='#999999',
    foreground_strong='#999999',
    foreground_subtle='#999999',
    opacity='.6',
    opacity_hover='.9',
    transition='400ms ease-in',
    colors=('#00AA00', '#AA0000', '#999999', '#AAAAAA', '#888888'))

def calculate_swiss_svg( acceptable_time_input:float, good_from_input:float, save_input:bool):
    test = df_kanton_delay.copy()
    not_acceptable_kanton_delay = df_kanton_delay.index[df_kanton_delay['delay'] > acceptable_time_input]
    grouped_kt_not_acceptable_delay = test.loc[not_acceptable_kanton_delay].groupby(['kanton'])['delay']
    quality = ((gp_kanton_delay.size() - grouped_kt_not_acceptable_delay.size()).fillna(gp_kanton_delay.size())/gp_kanton_delay.size()).sort_values() * 100

    kt_good = {}
    kt_bad = {}
    for k,v in quality.apply(float).to_dict().items():
        if v > good_from_input:
            kt_good[kt_name_conversion[k]] = v
        else:
            kt_bad[kt_name_conversion[k]] = 100-v


    ch_chart = pygal.maps.ch.Cantons(style = custom_style)
    ch_chart.title = 'Transport punctuality'
    ch_chart.add('Good (punctual)', kt_good)
    ch_chart.add('Bad (late)', kt_bad)

    swiss = ch_chart.render()

    IPy.display.display(IPy.display.SVG(swiss))
    
    if save_input:
        save_render.value = False
        open("./swiss_transport_punctuality.svg","wb").write(swiss)  

# widget objects
maximum_acceptable_time = widgets.FloatSlider(
    value=120.0,
    min=30,
    max=600,
    step=0.1,
    description='Acceptable delay [s] :',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)
good_from = widgets.FloatSlider(
    value=80,
    min=0,
    max=100,
    step=1,
    description='good from [percent] :',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f'
)
save_render = widgets.ToggleButton(
    value=False,
    description='Save render as svg',
    disabled=False,
    button_style='',
    tooltip='Description',
    icon='none'
)

# interact
interactions = widgets.interactive_output(
    calculate_swiss_svg,{'acceptable_time_input':maximum_acceptable_time,
                      'good_from_input':good_from,
                      'save_input':save_render
                     } )
widgetBox = widgets.VBox([maximum_acceptable_time, good_from, save_render])
IPy.display.display(widgetBox, interactions)

## Conclusion

### Problems encountered

#### Acquisition of datas
At the beginnning of the project we tried to directly fetch datas with [SBB API platfrom](https://developer.sbb.ch/home) using THe [Service Points](https://developer.sbb.ch/apis/servicepoints/information). We struggled a lot with this API because of the OAuth2 authentification. Finally we found everything we wanted in this [page](https://opentransportdata.swiss/fr/dataset/istdaten/) on [opentransportdata](https://opentransportdata.swiss/en/).

#### Relate a train station to its administration
It took us a lot of time to get in wich administration are the train stations, and it was crucial to show it on Swiss map.

### Limitations

#### Quantity of datas
- Although we can download datas for 50 days, it would be better to have datas for more than 1 year. It would increase accuracy.
Also datas are 'complete', what's cool, but that means they are large too. Indeed the size for 50 days is near to 25GB, which can take more than 100GB of RAM at the moment we load the csv.

### Ameliorations

#### Use more datas
- In our dataset we have discarded all datas that weren't from train, but we could also use all of them to compare them between different public transports.
- In our datas, we have the company of the train, and in the future, we will be able to make a comparison between all companies.

#### Improve usage of datas
- To show punctuality by kanton on the swiss map, we had to know in which administration is the train station. Therefore we used the name of the train station and tried to find the city's name with it. With this method we were only able to correlate for 20% of train station's name to an administration.

### Personal appreciation

We appreciated to do this project. We learn a lot about datas analysis in Python.<br>
Thanks to ours teacher and assistant for help and best regards.